In [1]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
import sys
import subprocess

# Upgrade related dependencies
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "hsfs", "hopsworks", "protobuf"])

  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)


CompletedProcess(args=['/opt/anaconda3/bin/python', '-m', 'pip', 'install', '--upgrade', 'hsfs', 'hopsworks', 'protobuf'], returncode=0)

In [5]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-04 18:23:02,625 INFO: Initializing external client
2025-03-04 18:23:02,625 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:23:03,211 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Fetching data from 2025-02-03 23:23:02.624103+00:00 to 2025-03-04 22:23:02.624103+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.45s) 


In [6]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 18:23:19,145 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:23:19,147 INFO: Initializing external client
2025-03-04 18:23:19,147 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:23:19,686 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641


In [7]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [8]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,255,1.0,2025-03-05 00:00:00+00:00
1,112,1.0,2025-03-05 00:00:00+00:00
2,218,1.0,2025-03-05 00:00:00+00:00
3,237,127.0,2025-03-05 00:00:00+00:00
4,57,0.0,2025-03-05 00:00:00+00:00
...,...,...,...
246,108,0.0,2025-03-05 00:00:00+00:00
247,71,1.0,2025-03-05 00:00:00+00:00
248,8,0.0,2025-03-05 00:00:00+00:00
249,168,0.0,2025-03-05 00:00:00+00:00


In [9]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-04 18:23:21,872 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:23:21,874 INFO: Initializing external client
2025-03-04 18:23:21,875 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:23:22,436 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641


In [10]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |█| Rows 251/251 | Elapsed Time: 00:00 | Remaining 


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215641/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)